In [47]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
import re

In [48]:
pd.set_option('display.max_colwidth', 1000)

In [49]:
PATH_TO_DATA = './data/ng'

In [50]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [51]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [86]:
data.head(5)

,content,keywords,summary,title,url,clean_text,clean_text_str,RTE,RTE10
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html,"[яблоко, фракция, задача, яблоко, привлечение, молодая, участие, выборы, деятельность, партия, яблоко, регион, руководство, деятельность, акция, уровень, борьба, призыв, военный, профессия, обязанность, молодая, государство, долг, мнение, специалист, область, политика, алексей, арбатов, переход, армия, военный, бюджет, уровень, яблоко, акция, освобождение, политзаключённый, вмешательство, россия, политика, украина, активист, виктор, петрунин, москва, семья, взгляд, виктор, детство, политика, виктор, власть, крым, война, украина, убийство, борис, немцов, сторона, школа, сила, смена, власть, россия, дарья, новичкова, влияние, нея, преподаватель, обществознание, ситуация, страна, искренность, заинтересованность, вопрос, дарья, материал, сми, наблюдение, позиция, образ, команда, единомышленник, пример, ровесник, дарья, яблоко, яблоко, клуб, рамка, лекция, кинопоказ, тема, наступление, консерватизм, обсуждение, альтернатива, ситуация, ...]",яблоко фракция задача яблоко привлечение молодая участие выборы деятельность партия яблоко регион руководство деятельность акция уровень борьба призыв военный профессия обязанность молодая государство долг мнение специалист область политика алексей арбатов переход армия военный бюджет уровень яблоко акция освобождение политзаключённый вмешательство россия политика украина активист виктор петрунин москва семья взгляд виктор детство политика виктор власть крым война украина убийство борис немцов сторона школа сила смена власть россия дарья новичкова влияние нея преподаватель обществознание ситуация страна искренность заинтересованность вопрос дарья материал сми наблюдение позиция образ команда единомышленник пример ровесник дарья яблоко яблоко клуб рамка лекция кинопоказ тема наступление консерватизм обсуждение альтернатива ситуация экономика участник стол мероприятие место дискуссия участие люба гость яблоко повестка ноябрь прошлое карелия акция свобода ильдар дадин ряд колония проц...,"[яблоко, акция, жизнь, власть, виктор, федеральный уровень, наши активисты, московские активисты, участие, россия, политика, молодёжь, дарья, года, выборы, активисты, оборонная политика алексея арбатова, главная предвыборная кампания страны, 10 лет лишения свободы, убийство бориса немцова, свобода ильдара дадина, самые разные тема, самые больший репрессии, самая невыгодная позиция, привлечение молодых людей, обязательные воинский призыв, ноябрь прошлого года, многие злободневные вопросы, количество молодых людей, все возможные усилие, внутренняя политика украины, активная гражданская позиция, целый год, функция организатора, учебные заведения, уголовные дела, уголовное дело, тяжёлый ущерб, театрализованная акция, такая ответственность, судебный процесс, сторона силовиков, социально-либера

In [52]:
from stop_words import get_stop_words

STW = get_stop_words('russian')
new = ['это', 'который', 'один', 'что', 'мой', 'свой']
SW = STW + new

In [58]:
def clean(words):
    
    clean = []
    
    for word in words:
        p = morph.parse(word)[0]
        if 'NOUN' in p.tag:
            clean.append(p.normal_form)
    return clean

In [59]:
def process(text): #токенизируем
    
    global SW
    
    word = re.compile(r'[А-яё]{2,}|[A-z]{2,}')

    text = text.lower()
    words = word.findall(text)
    
    processed_text = clean(words)
    
    final_text = [word for word in processed_text if word not in SW]
    
    return final_text

In [60]:
from stop_words import get_stop_words

STW = get_stop_words('russian')
new = ['это', 'который', 'один', 'что', 'мой', 'свой']
SW = STW + new

In [61]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

### 1. Другая нормализация

In [62]:
data['clean_text'] = data['content'].apply(process)

In [63]:
data['clean_text_str'] = data['clean_text'].apply(' '.join)

In [64]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [65]:
tfidf.fit(data['clean_text_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [66]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [67]:
texts_vectors = tfidf.transform(data['clean_text_str'])

In [68]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [69]:
print(keywords[:3])

[['яблоко', 'активист', 'акция', 'дарья', 'деятельность', 'политика', 'виктор', 'тимур', 'репрессия', 'выборы'], ['миллиард куб', 'куб', 'газпром', 'газа', 'миллиард', 'добыча', 'добыча газа', 'холдинг', 'производитель', 'прогноз'], ['роман', 'книга', 'жанр', 'стихотворение', 'читатель', 'перевод', 'поэзия', 'произведение', 'туманов', 'реквием']]


In [70]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


### 2. RUTERMEXTRACT

In [76]:
from rutermextract import TermExtractor

In [77]:
def get_term_candidates(text):
        
    terms = []
    term_extractor = TermExtractor()

    for term in term_extractor(text):
        terms.append(term.normalized)
    return terms

In [78]:
data['RTE'] = data['content'].apply(get_term_candidates)

/opt/conda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [85]:
data['RTE10'] = data['RTE'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)])

In [87]:
evaluate(data['keywords'], data['RTE10'])

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


### 3. Частотный словарь

In [ ]:
### не смогла реализовать